In [1]:
import time
import pandas as pd
import numpy as np
import os
import json
import string

In [2]:
df_rev = pd.read_pickle('results/reviews_hu.p')

In [3]:
df_rev.date = df_rev.date.astype(np.datetime64)
df_rev['ind'] = list(range(0, len(df_rev)))

In [4]:
df_rev.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,review_tokens,sent_score_compound,sent_score_net,review_length,punc_count,avg_word_len,ind
0,f5O7v_X_jCg2itqacRfxhg,0,2017-10-12,0,kS4hrhEScwB9V5JATYjvVQ,5,Sansotei serves some top notch ramen. They tak...,0,hxqo4NyJFfeOmuoVi--s1A,"[sansotei, serves, some, top, notch, ramen, .,...",-0.2500,0.000000,108,0.009174,4.111111,0
1,Edr6SafmQrZa6CcFuItoqg,0,2014-06-03,0,U04YgYr4PiGUxB46J4xapg,2,Limited vegetarian options. Ordered Greek sala...,0,B7Fh30KQT1xPSGt_pIhRRA,"[limited, vegetarian, options, ., ordered, gre...",-0.5719,-0.029412,34,0.055556,4.911765,1
2,7xA6iSP0Ndn08tpBFQtUKA,0,2017-05-22,0,YDJDfKnx6VpMMo4EBxycGg,1,Non-existened service! The waiter did not eve...,0,FCtoTo9zSH1cSAkascfEHw,"[non-existened, service, !, the, waiter, did, ...",-0.8948,-0.011236,89,0.072917,4.404494,2
3,SmizR7MLt-558FJJQRBBoQ,1,2011-06-01,1,2Hk7DNwu3rb2jKHaFfPyCA,4,It might be a bit unfair to review Globe so ea...,1,YHWsLBS8jzZiPjKHMFOaAA,"[it, might, be, a, bit, unfair, to, review, gl...",0.9982,0.028571,700,0.012694,4.305714,3
4,iKMLsX1Je7P3wAOEc9scDg,0,2011-11-07,0,D2TcNaPqXxDGQ6T0n-vHXw,4,I have been itching to get to Origin for month...,4,YHWsLBS8jzZiPjKHMFOaAA,"[i, have, been, itching, to, get, to, origin, ...",0.9994,0.036066,915,0.022436,4.093989,4


In [5]:
# rolling window size = 2 years 730days
aggregator = {'ind': np.size,
              'stars': np.mean, 
              'funny':np.sum, 
              'cool':np.sum, 
              'useful': np.sum,
              'sent_score_compound': np.mean,
              'sent_score_net':np.mean, 
              'review_length': np.mean,
              'punc_count':np.mean, 
              'avg_word_len': np.mean
               }

bus_rev_res = df_rev.sort_values('date').set_index('date').groupby(['business_id']).rolling(window='730d').agg(aggregator)
user_rev_res = df_rev.sort_values('date').set_index('date').groupby(['user_id']).rolling(window='730d').agg(aggregator)

In [6]:
bus_rev_res.columns = ['count_review', 'avg_stars', 'count_funny', 'count_cool', 'count_useful', 'avg_sent_score_compound',
       'avg_sent_score_net', 'avg_review_length', 'avg_punc_count', 'avg_word_len']
user_rev_res.columns = ['count_review', 'avg_stars', 'count_funny', 'count_cool', 'count_useful', 'avg_sent_score_compound',
       'avg_sent_score_net', 'avg_review_length', 'avg_punc_count', 'avg_word_len']

In [7]:
bus_rev_res = pd.DataFrame(bus_rev_res.to_records())
user_rev_res = pd.DataFrame(user_rev_res.to_records())

In [8]:
bus_rev_res.to_pickle('results/bus_rev_res_hu.p')
user_rev_res.to_pickle('results/user_rev_res_hu.p')

In [ ]:
# bus_rev_res = pd.read_pickle('results/bus_rev_res_hu.p')
# user_rev_res = pd.read_pickle('results/user_rev_res_hu.p')

In [9]:
bus_rev_res = bus_rev_res.drop_duplicates(['business_id', 'date'], keep='last')
user_rev_res = user_rev_res.drop_duplicates(['user_id', 'date'], keep='last')

In [11]:
bus_cols = bus_rev_res.columns.tolist()[2:]
user_cols = user_rev_res.columns.tolist()[2:]

In [12]:
for col in bus_cols:
    bus_rev_res[col] = bus_rev_res.groupby('business_id')[col].shift(1)
for col in user_cols:
    user_rev_res[col] = user_rev_res.groupby('user_id')[col].shift(1)

In [13]:
bus_rev_res.head()

,business_id,date,count_review,avg_stars,count_funny,count_cool,count_useful,avg_sent_score_compound,avg_sent_score_net,avg_review_length,avg_punc_count,avg_word_len
0,--DaPTJW3-tB1vP-PfdTEg,2012-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,--DaPTJW3-tB1vP-PfdTEg,2012-06-14,1.0,4.000000,0.0,0.0,0.0,0.986900,0.082803,157.0,0.042683,4.000000
2,--DaPTJW3-tB1vP-PfdTEg,2012-11-11,2.0,4.000000,1.0,2.0,1.0,0.941900,0.090182,99.0,0.044597,4.073171
3,--DaPTJW3-tB1vP-PfdTEg,2013-04-07,3.0,4.333333,1.0,2.0,1.0,0.932067,0.087899,94.0,0.037483,4.259098
4,--DaPTJW3-tB1vP-PfdTEg,2013-07-06,4.0,4.250000,1.0,2.0,1.0,0.934850,0.104987,78.5,0.028113,4.202136


In [14]:
user_rev_res.head()

,user_id,date,count_review,avg_stars,count_funny,count_cool,count_useful,avg_sent_score_compound,avg_sent_score_net,avg_review_length,avg_punc_count,avg_word_len
0,--56y1InAvNoQOD6YYrhVQ,2016-08-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,--7gjElmOrthETJ8XqzMBw,2014-11-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,--7gjElmOrthETJ8XqzMBw,2018-05-24,1.0,2.0,0.0,0.0,0.0,0.5106,0.041667,48.0,0.04,4.708333
3,--Br-QsbO9ad5GbZxVGxaw,2016-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,--BumyUHiO_7YsHurb9Hkw,2017-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
bus_rev_res.to_pickle('results/bus_rev_res__offset_hu.p')
user_rev_res.to_pickle('results/user_rev_res__offset_hu.p')